In [1]:
import json
import os
import re
import time
from datetime import datetime
from datetime import date
from time import sleep
import click
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_auth
from dash import Dash,html,Input, dcc, Output
# from dash import dcc
import plotly.express as px
import plotly
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
from pandas.api.types import CategoricalDtype
import json
import os
import re
import time
from datetime import datetime
from datetime import date
from time import sleep
import pandas as pd
import numpy as np
import time
import datetime
from datetime import date
from time import sleep
from openpyxl import load_workbook

/var/folders/8l/zp31g9ld30qbb19mnryb_bjs36vbxs/T/ipykernel_51920/4216134235.py:14: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/Users/linzhidian/opt/anaconda3/lib/python3.9/site-packages/dash_auth/plotly_auth.py:17: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [11]:
staff_list = pd.read_csv('10月人员维度.csv')
staff_list

,员工姓名,员工组,工作地点,业务线,员工所属部门,实际人天,预估人天,理论人天,实际-预估Gap,预估填报率,实际-理论Gap,理论填报率,预估填报,理论填报
0,严志田,正式员工,北京,DX-TY,智慧体育业务,0.0,18,19,-18.0,0.0,-19.0,0.0,不满,不满
1,徐小凤,正式员工,成都,DX-SKU,新零售业务,14.0,19,19,-5.0,73.7,-5.0,73.7,不满,不满
2,徐建兴,正式员工,成都,IRDC,创新与赋能中心,13.0,25,19,-12.0,52.0,-6.0,68.4,不满,不满
3,戴云川,正式员工,成都,SX-IBG,中东云平台,14.0,34,19,-20.0,41.2,-5.0,73.7,不满,不满
4,文康,正式员工,北京,SI,亚太研发中心,12.0,19,19,-7.0,63.2,-7.0,63.2,不满,不满
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,刘鑫,正式员工,上海,SX-IBG,中东云平台,27.0,35,19,-8.0,77.1,8.0,142.1,不满,超载
115,郑柏屹,正式员工,深圳,IR,智慧娱乐,19.6,22,19,-2.4,89.1,0.6,103.2,合理,合理
116,魏琳月,正式员工,阿布扎比,SX-IBG,中东云平台,21.0,32,19,-11.0,65.6,2.0,110.5,不满,合理
117,林斋帆,正式员工,深圳,AIOT,海外智能终端与应用,21.0,19,19,2.0,110.5,2.0,110.5,合理,合理


In [14]:
业务线汇总 = staff_list.iloc[0:-1, :]
业务线汇总 = 业务线汇总.replace('inf%', None)
预估业务线汇总 = 业务线汇总[["业务线", "员工数", "预估人天"]].rename(columns={"预估人天": "总工时"})
预估业务线汇总['工时类别'] = 预估业务线汇总['业务线'].apply(lambda row: '预估')
实际业务线汇总 = 业务线汇总[["业务线", "员工数", "实际人天"]].rename(columns={"实际人天": "总工时"})
实际业务线汇总['工时类别'] = 实际业务线汇总['业务线'].apply(lambda row: '实际')
理论业务线汇总 = 业务线汇总[["业务线", "员工数", "理论人天"]].rename(columns={"理论人天": "总工时"})
理论业务线汇总['工时类别'] = 理论业务线汇总['业务线'].apply(lambda row: '理论')
预计实际业务线汇总 = 预估业务线汇总.append(实际业务线汇总).append(理论业务线汇总).reset_index(drop=True)
预计实际业务线汇总

KeyError: "['员工数'] not in index"

In [20]:
staff_list

,员工姓名,员工组,工作地点,业务线,员工所属部门,实际人天,预估人天,理论人天,实际-预估Gap,预估填报率,实际-理论Gap,理论填报率,预估填报,理论填报
0,严志田,正式员工,北京,DX-TY,智慧体育业务,0.0,18,19,-18.0,0.0,-19.0,0.0,不满,不满
1,徐小凤,正式员工,成都,DX-SKU,新零售业务,14.0,19,19,-5.0,73.7,-5.0,73.7,不满,不满
2,徐建兴,正式员工,成都,IRDC,创新与赋能中心,13.0,25,19,-12.0,52.0,-6.0,68.4,不满,不满
3,戴云川,正式员工,成都,SX-IBG,中东云平台,14.0,34,19,-20.0,41.2,-5.0,73.7,不满,不满
4,文康,正式员工,北京,SI,亚太研发中心,12.0,19,19,-7.0,63.2,-7.0,63.2,不满,不满
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,刘鑫,正式员工,上海,SX-IBG,中东云平台,27.0,35,19,-8.0,77.1,8.0,142.1,不满,超载
115,郑柏屹,正式员工,深圳,IR,智慧娱乐,19.6,22,19,-2.4,89.1,0.6,103.2,合理,合理
116,魏琳月,正式员工,阿布扎比,SX-IBG,中东云平台,21.0,32,19,-11.0,65.6,2.0,110.5,不满,合理
117,林斋帆,正式员工,深圳,AIOT,海外智能终端与应用,21.0,19,19,2.0,110.5,2.0,110.5,合理,合理


In [19]:
预估业务线汇总 = staff_list[["业务线", "员工组", "预估人天"]].rename(columns={"预估人天": "总工时"})
预估业务线汇总['工时类别'] = 预估业务线汇总['业务线'].apply(lambda row: '预估')
实际业务线汇总 = staff_list[["业务线", "员工组", "实际人天"]].rename(columns={"实际人天": "总工时"})
实际业务线汇总['工时类别'] = 实际业务线汇总['业务线'].apply(lambda row: '实际')
理论业务线汇总 = staff_list[["业务线", "员工组", "理论人天"]].rename(columns={"理论人天": "总工时"})
理论业务线汇总['工时类别'] = 理论业务线汇总['业务线'].apply(lambda row: '理论')
预计实际业务线汇总 = 预估业务线汇总.append(实际业务线汇总).reset_index(drop=True).append(理论业务线汇总).reset_index(drop=True)
预计实际业务线汇总 = 预计实际业务线汇总.groupby(["业务线", "员工组","工时类别"]).agg({"总工时":'sum'}).reset_index()

预计实际业务线汇总

/var/folders/8l/zp31g9ld30qbb19mnryb_bjs36vbxs/T/ipykernel_51920/3850690194.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  预计实际业务线汇总 = 预估业务线汇总.append(实际业务线汇总).reset_index(drop=True).append(理论业务线汇总).reset_index(drop=True)


,业务线,员工组,工时类别,总工时
0,AIOT,外包员工,实际,14.0
1,AIOT,外包员工,理论,19.0
2,AIOT,外包员工,预估,19.0
3,AIOT,实习生,实际,14.0
4,AIOT,实习生,理论,19.0
5,AIOT,实习生,预估,19.0
6,AIOT,正式员工,实际,157.0
7,AIOT,正式员工,理论,209.0
8,AIOT,正式员工,预估,209.0
9,DX-SKU,外包员工,实际,13.2


In [ ]:
人均预计实际业务线汇总 = 预计实际业务线汇总.copy()